In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

--- Execução do Notebook Silver ---

In [2]:
# Caminho para o arquivo Bronze
bronze_file_path = '../data/bronze/dados_brutos.csv'

In [3]:
# Carregar dados da camada Bronze
try:
    if not os.path.exists(bronze_file_path):
        print(f"Erro: Arquivo '{bronze_file_path}' não encontrado.")
        df_silver = pd.DataFrame()
    else:
        df_silver = pd.read_csv(bronze_file_path)
        print(f"Dados originais carregados: {df_silver.shape}")
except Exception as e:
    print(f"Ocorreu um erro ao carregar o arquivo '{bronze_file_path}': {e}")
    df_silver = pd.DataFrame()

Dados originais carregados: (325753, 31)


In [4]:
# Verificar se o DataFrame não está vazio antes de prosseguir com as transformações
if not df_silver.empty:
    # ==========================================
    # TRANSFORMAÇÃO 1: Remover Duplicatas
    # ==========================================
    linhas_antes = len(df_silver)
    df_silver = df_silver.drop_duplicates().copy()
    linhas_depois = len(df_silver)
    print(f"Duplicatas removidas: {linhas_antes - linhas_depois}")

    # ==========================================
    # TRANSFORMAÇÃO 2: Tratar Valores Nulos
    # ==========================================
    colunas_regiao = ['State', 'District ']
    df_silver = df_silver.dropna(subset=colunas_regiao).copy()

    # Preencher valores numéricos com a mediana
    colunas_numericas = df_silver.select_dtypes(include=[np.number]).columns
    df_silver[colunas_numericas] = df_silver[colunas_numericas].fillna(df_silver[colunas_numericas].median())

    # Preencher valores categóricos restantes com 'Desconhecido'
    colunas_texto = df_silver.select_dtypes(include=['object']).columns
    df_silver[colunas_texto] = df_silver[colunas_texto].fillna('Desconhecido')
    print(f"Tratamento de nulos concluído.")

    # ==========================================
    # TRANSFORMAÇÃO 3: Padronizar Valores E CRIAR COLUNA-CHAVE (MELHORIA 2)
    # ==========================================
    # 1. Padronizar colunas de texto (strip/upper)
    colunas_para_padronizar = ['State', 'District ', 'Crop', 'Season']
    for coluna in colunas_para_padronizar:
        if coluna in df_silver.columns:
            # Padroniza antes do filtro de cultura
            df_silver[coluna] = df_silver[coluna].astype(str).str.strip().str.upper().copy()
    print("Colunas de texto padronizadas (strip/upper).")

    # 2. Renomear 'District ' (com espaço) para 'District' (sem espaço)
    if 'District ' in df_silver.columns:
        df_silver.rename(columns={'District ': 'District'}, inplace=True)
        print("Renomeada a coluna 'District ' para 'District'.")

    # Filtra linhas onde a coluna 'State' (agora em maiúsculo) tem o valor literal 'STATE'.
    linhas_antes_filtro_header = len(df_silver)
    df_silver = df_silver[df_silver['State'] != 'STATE'].copy()
    print(f"Removida {linhas_antes_filtro_header - len(df_silver)} linha(s) de 'header como dado'.")

    # 3. Criar a coluna combinada 'State District'
    if 'State' in df_silver.columns and 'District' in df_silver.columns:
        df_silver['State District'] = (df_silver['State'] + ' - ' + df_silver['District']).copy()
        print("Coluna 'State District' criada com sucesso.")

    # Remover colunas State e District
    df_silver.drop(columns=['State', 'District'], inplace=True)
    print("Colunas 'State' e 'District' removidas.")

    # ==========================================
    # TRANSFORMAÇÃO 4: Simplificando o escopo de colheita/Crop (Filtro)
    # ==========================================
    # Agora o filtro é aplicado APÓS a padronização para UPPER()
    valores_desejados = ['MAIZE','RICE','WHEAT','BARLEY']
    if 'Crop' in df_silver.columns:
        mascara = df_silver['Crop'].isin(valores_desejados)
        df_silver = df_silver[mascara].copy()
        print(f"Dados filtrados para as colheitas desejadas. Novo shape: {df_silver.shape}")
    else:
        print("Aviso: A coluna 'Crop' não foi encontrada. A Transformação 4 foi ignorada.")

    # ==========================================
    # TRANSFORMAÇÃO 5: Remover Registros de Produção Zero (e Yield Zero)
    # ==========================================
    print(f"\nRemovendo registros com 'Area' ou 'Yield' zerados...")
    linhas_antes_filtro_zero = len(df_silver)

    # Filtro 1: Remove registros onde a área plantada é 0 (dado inválido)
    if 'Area' in df_silver.columns:
        df_silver = df_silver[df_silver['Area'] > 0].copy()

    # Filtro 2: Remove registros onde o rendimento é 0 (dado inútil para análise)
    if 'Yield' in df_silver.columns:
        df_silver = df_silver[df_silver['Yield'] > 0].copy()

    linhas_depois_filtro_zero = len(df_silver)
    print(f"Linhas removidas pelos filtros de zero: {linhas_antes_filtro_zero - linhas_depois_filtro_zero}")
    print(f"Linhas restantes após o filtro de zeros: {len(df_silver)}")


    # ==========================================
    # VALIDAÇÕES BÁSICAS (Para fins de log)
    # ==========================================
    print("\n=== VALIDAÇÕES DE QUALIDADE ===")
    if not df_silver.empty:
        total_celulas = df_silver.shape[0] * df_silver.shape[1]
        celulas_preenchidas = df_silver.count().sum()
        completude = (celulas_preenchidas / total_celulas) * 100
        print(f"Completude: {completude:.2f}%")

        duplicatas = df_silver.duplicated().sum()
        unicidade = ((len(df_silver) - duplicatas) / len(df_silver)) * 100
        print(f"Unicidade: {unicidade:.2f}% (Total de {duplicatas} duplicatas restantes)")
    else:
        print("Validação ignorada: DataFrame está vazio após as transformações.")

    # Adicionar informações de processamento
    df_silver['data_processamento'] = datetime.now()

    # Salvar na camada Silver
    silver_file_path = '../data/silver/dados_limpos.csv'
    os.makedirs(os.path.dirname(silver_file_path), exist_ok=True)
    df_silver.to_csv(silver_file_path, index=False)
    print(f"\nDados limpos salvos: {df_silver.shape}")


Duplicatas removidas: 0
Tratamento de nulos concluído.
Colunas de texto padronizadas (strip/upper).
Renomeada a coluna 'District ' para 'District'.
Removida 0 linha(s) de 'header como dado'.
Coluna 'State District' criada com sucesso.
Colunas 'State' e 'District' removidas.
Dados filtrados para as colheitas desejadas. Novo shape: (56086, 30)

Removendo registros com 'Area' ou 'Yield' zerados...
Linhas removidas pelos filtros de zero: 404
Linhas restantes após o filtro de zeros: 55682

=== VALIDAÇÕES DE QUALIDADE ===
Completude: 100.00%
Unicidade: 100.00% (Total de 0 duplicatas restantes)

Dados limpos salvos: (55682, 31)


In [5]:
print(df_silver)

         Crop  YEAR  Season     Area  Production  Yield   JAN   FEB   MAR  \
355     MAIZE  2007    RABI   230.81       879.0   3.81   1.7  36.7  35.2   
356     MAIZE  2008  SUMMER   239.00       641.0   2.68  18.4  19.3  41.2   
357     MAIZE  2009  SUMMER   163.00       405.0   2.48  12.0  12.0  14.2   
358     MAIZE  2010    RABI     3.84        18.0   4.74   7.5  17.0  14.0   
359     MAIZE  2011    RABI     4.00         3.0   0.68   6.8  25.8  22.4   
...       ...   ...     ...      ...         ...    ...   ...   ...   ...   
325748  WHEAT  2014    RABI  1622.00      3663.0   2.26  19.2  27.4  36.1   
325749  WHEAT  2015    RABI   855.00      1241.0   1.45  17.9  25.6  36.1   
325750  WHEAT  2016    RABI  1366.00      2415.0   1.77  17.6  26.2  34.4   
325751  WHEAT  2017    RABI  1052.00      2145.0   2.04  18.2  26.6  33.0   
325752  WHEAT  2018    RABI   833.00      2114.0   2.54  18.5  26.5  32.1   

         APR  ...  Jun-Sep  Oct-Dec  TEMP_ANNUAL  TEMP_JAN_FEB  TEMP_MAR_MA